## This script is used to clean the database. Do not run if you don't know what you are doing.

In [1]:
import sqlite3
import pandas as pd

# Connect to the database
db_path = '/Users/yuanlu/Code/youtube-top-10000-channels/data/videoamigo-processed.db'
conn = sqlite3.connect(db_path)

# Print tables in database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in database:")
for table in tables:
    print(f"- {table[0]}")


Tables in database:
- unique_youtube_channel_urls


In [3]:
# get column names
cursor.execute("PRAGMA table_info(unique_youtube_channel_urls)")
column_names = [row[1] for row in cursor.fetchall()]
print(f"Column names in unique_youtube_channel_urls table: {column_names}")


Column names in unique_youtube_channel_urls table: ['Channel_Handle', 'Channel_ID', 'Subscribers', 'YouTube_Channel_URL']


In [4]:
# Get Youtube_Channel_URL column from unique_youtube_channel_urls table and save it to a csv file
cursor.execute("SELECT Youtube_Channel_URL FROM unique_youtube_channel_urls")
youtube_channel_urls = cursor.fetchall()
df = pd.DataFrame(youtube_channel_urls, columns=["Youtube_Channel_URL"])
df.to_csv("youtube_channel_urls.csv", index=False)


In [29]:
# Get size of combined_data table
cursor.execute("SELECT COUNT(*) FROM combined_data")
row_count = cursor.fetchone()[0]
print(f"Size of combined_data table: {row_count:,} rows")


Size of combined_data table: 4,935,482 rows


In [30]:
# Get number of unique YouTube_Channel_URL
cursor.execute("SELECT COUNT(DISTINCT Youtube_Channel_URL) FROM combined_data")
unique_count = cursor.fetchone()[0]
print(f"Number of unique YouTube_Channel_URL: {unique_count}")

Number of unique YouTube_Channel_URL: 155976


In [32]:
# Percentage of unique youtube_channel_urls
print(f"Percentage of unique youtube_channel_urls: {unique_count / row_count * 100:.2f}%")


Percentage of unique youtube_channel_urls: 3.16%


In [33]:
# Drop the unique_youtube_channel_urls table if it exists
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS unique_youtube_channel_urls")
    conn.commit()

In [34]:
# Create a new table with unique youtube_channel_urls, it should have the same columns as combined_data. Make sure youtube_channel_url is unique.
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE unique_youtube_channel_urls AS 
        SELECT *
        FROM combined_data
        GROUP BY youtube_channel_url
    """)
    conn.commit()

## unique_youtube_channel_urls table

In [37]:
#  Connect to unique_youtube_channel_urls table
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [38]:
# Get size of unique_youtube_channel_urls table
cursor.execute("SELECT COUNT(*) FROM unique_youtube_channel_urls")
row_count = cursor.fetchone()[0]
print(f"Size of unique_youtube_channel_urls table: {row_count:,} rows")


Size of unique_youtube_channel_urls table: 155,976 rows


In [39]:
# drop combined_data table
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute("DROP TABLE combined_data")
    conn.commit()

In [40]:
# in the table unique_youtube_channel_urls, drop the columns called "Rank", "source_file"
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute("ALTER TABLE unique_youtube_channel_urls DROP COLUMN Rank")
    cursor.execute("ALTER TABLE unique_youtube_channel_urls DROP COLUMN source_file")
    conn.commit()


In [41]:
# how many rows in unique_youtube_channel_urls have Youtube_Channel_URL start with https://www.youtube.com/channel/UC
cursor.execute("SELECT COUNT(*) FROM unique_youtube_channel_urls WHERE Youtube_Channel_URL LIKE 'https://www.youtube.com/channel/%'")
rows_start_with_uc = cursor.fetchone()[0]
print(f"Number of rows in unique_youtube_channel_urls that start with https://www.youtube.com/channel/: {rows_start_with_uc}")


Number of rows in unique_youtube_channel_urls that start with https://www.youtube.com/channel/: 120387


In [42]:
# how many rows in unique_youtube_channel_urls have Youtube_Channel_URL not start with https://www.youtube.com/channel/UC
cursor.execute("SELECT COUNT(*) FROM unique_youtube_channel_urls WHERE Youtube_Channel_URL NOT LIKE 'https://www.youtube.com/channel/%'")
rows_not_start_with_uc = cursor.fetchone()[0]
print(f"Number of rows in unique_youtube_channel_urls that do not start with https://www.youtube.com/channel/: {rows_not_start_with_uc}")


Number of rows in unique_youtube_channel_urls that do not start with https://www.youtube.com/channel/: 35589


In [44]:
# Update URLs for rows where channel_id is valid (22 characters) but URL format is incorrect
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute("""
        UPDATE unique_youtube_channel_urls
        SET Youtube_Channel_URL = 'https://www.youtube.com/channel/UC' || channel_id
        WHERE Youtube_Channel_URL NOT LIKE 'https://www.youtube.com/channel/%'
        AND length(channel_id) = 22
    """)
    conn.commit()

# Verify the changes
cursor.execute("SELECT COUNT(*) FROM unique_youtube_channel_urls WHERE Youtube_Channel_URL NOT LIKE 'https://www.youtube.com/channel/%'")
remaining_incorrect = cursor.fetchone()[0]
print(f"Remaining rows with incorrect URL format: {remaining_incorrect}")

Remaining rows with incorrect URL format: 17374


In [45]:
# for the rows in unique_youtube_channel_urls have Youtube_Channel_URL start with https://www.youtube.com/@, find the top 10 with the most characters in Channel_Handle
cursor.execute("SELECT * FROM unique_youtube_channel_urls WHERE Youtube_Channel_URL LIKE 'https://www.youtube.com/@%' ORDER BY LENGTH(Channel_Handle) DESC LIMIT 10")
top_10_with_most_characters = cursor.fetchall()
print(f"Top 10 rows with the most characters in Channel_Handle where Youtube_Channel_URL starts with https://www.youtube.com/@:")
for row in top_10_with_most_characters:
    print(row)

Top 10 rows with the most characters in Channel_Handle where Youtube_Channel_URL starts with https://www.youtube.com/@:
('à¸„à¸£à¸\xadà¸šà¸„à¸£à¸±à¸§à¸‚à¹ˆà¸²à¸§3', 'à¸„à¸£à¸\xadà¸šà¸„à¸£à¸±à¸§à¸‚à¹ˆà¸²à¸§3', 2570000, 'https://www.youtube.com/@à¸„à¸£à¸\xadà¸šà¸„à¸£à¸±à¸§à¸‚à¹ˆà¸²à¸§3')
('Amazon Prime Video India', 'Amazon Prime Video India', 25300000, 'https://www.youtube.com/@Amazon Prime Video India')
('Friedrich-Ebert-Stiftung', 'Friedrich-Ebert-Stiftung', 18900, 'https://www.youtube.com/@Friedrich-Ebert-Stiftung')
('Pravoslavlje_Orthodox', 'Pravoslavlje_Orthodox', 20900, 'https://www.youtube.com/@Pravoslavlje_Orthodox')
('2010floridastatefair', '2010floridastatefair', 54, 'https://www.youtube.com/@2010floridastatefair')
('20thcenturyfoxfrance', '20thcenturyfoxfrance', 345000, 'https://www.youtube.com/@20thcenturyfoxfrance')
('925thebeatofmontreal', '925thebeatofmontreal', 2990, 'https://www.youtube.com/@925thebeatofmontreal')
('AASchoolArchitecture', 'AASchoolArchitecture', 57300, 

In [47]:
# how many rows have subscribers less than 100?
cursor.execute("SELECT COUNT(*) FROM unique_youtube_channel_urls WHERE Subscribers < 100")
subscribers_less_than_100 = cursor.fetchone()[0]
print(f"Number of rows with subscribers less than 100: {subscribers_less_than_100}")


Number of rows with subscribers less than 100: 17530


In [48]:
# how many rows have subscribers less than 10?
cursor.execute("SELECT COUNT(*) FROM unique_youtube_channel_urls WHERE Subscribers < 10")
subscribers_less_than_10 = cursor.fetchone()[0]
print(f"Number of rows with subscribers less than 10: {subscribers_less_than_10}")

Number of rows with subscribers less than 10: 5484
